# Taller: Redes convolucionales

© Deep Learning Indaba. Apache License 2.0.

## Introducción
En este taller, cubriremos los conceptos básicos de las redes neuronales convolucionales, o "ConvNets". ConvNets se inventaron a fines de la década de 1980 / principios de la década de 1990, y han tenido un gran éxito, especialmente con la visión computacional, aunque también se han utilizado con mucho éxito en las líneas de procesamiento de voz y, más recientemente, en la traducción automática.

## Objetivos de aprendizaje
* Ser capaz de explicar qué hace una capa convolucional(convolutional layer) y cómo es diferente de una capa completamente conectada(fully-connected layer).
* Comprender las ventajas, desventajas y las suposiciones  que se hacen que cuando se utiliza arquitecturas convolucionales
* Ser capaz de construir una arquitectura convolucional usando Tensorflow 2.0 y Keras Layers.
* Ser capaz de usar Keras para entrenar un modelo en un conjunto de datos.
* Implementar la normalización por lotes(batch normalization) o una red residual muy pequeña.

## Ejecución en GPU
Para esta práctica,se necesitará usar una GPU para acelerar el entrenamiento. Para hacer esto, vaya al menú "Runtime" en Colab, seleccione "Change runtime type" y luego en el menú emergente, elija "GPU" en el cuadro "Hardware accelerator".
¡Esto es todo lo que necesita hacer, Colab y Tensorflow se encargarán del resto!

In [0]:
#@title Imports (RUN ME!) { display-mode: "form" }

!pip install tensorflow-gpu==2.0.0-beta0 > /dev/null 2>&1
!pip -q install pydot_ng > /dev/null 2>&1
!pip -q install graphviz > /dev/null 2>&1
!apt install graphviz > /dev/null 2>&1

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

print("TensorFlow executing eagerly: {}".format(tf.executing_eagerly()))

## Arquitecturas convolucionales
Al modelar una imagen usando una red Feed-forward regular, encontramos que el número de parámetros del modelo crece exponencialmente. 

**PREGUNTA**: ¿Cuántos parámetros habría en una red de alimentación con 2 capas ocultas que consisten en 512 y 256 neuronas respectivamente, un tamaño de salida de 10 y una imagen de entrada de forma [32, 32, 3]? (Tenga en cuenta que representamos cada píxel en una imagen en color usando tres números reales para los valores Rojo, Verde y Azul - [llamados "canales"] (https://www.quora.com/What-do-channels-refer -a-en-una-red-neuronal convolucional) - de ahí la forma 32x32 **x3**.)

ConvNets aborda este problema de parámetros del modelo explotando la estructura en las entradas a la red (en particular, suponiendo que la entrada es en **volumen 3D** , que se aplica a las imágenes, por ejemplo, donde las 3 dimensiones consisten en tres Canales RGB). Las dos diferencias clave entre ConvNet y la red Feed-forward son:

* ConvNets tienen neuronas que se organizan en 3 dimensiones: ancho, alto, profundidad. Tenga en cuenta que *profundidad* aquí significa canales, es decir, la profundidad del volumen de entrada, ¡no la profundidad de una red neuronal profunda!
* Las neuronas en cada capa solo están conectadas a una pequeña región de la capa anterior.

**PREGUNTA**: ¿Qué compensación cree que ConvNet hace para la reducción de memoria requerida por menos parámetros?

En general, la arquitectura ConvNet se compone de diferentes tipos de capas, las más comunes son las capas convolucionales, las capas de pooling y las capas totalmente conectadas(fully-connected) que encontramos en la última práctica.

### Lectura adicional opcional: el surgimiento de arquitecturas convolucionales profundas

Las arquitecturas de ConvNet fueron clave para el tremendo éxito del aprendizaje profundo en visión artificial. En particular, el primer modelo de aprendizaje profundo para ganar la competencia ImageNet en 2012 se llamó AlexNet (debido s Alex Krizhevsky, uno de sus inventores). Tenía 5 capas convolucionales seguidas de 3 capas completamente conectadas. Los ganadores posteriores incluyeron GoogLeNet y ResNet. Si tiene curiosidad, eche un vistazo a [este enlace] (https://medium.com/towards-data-science/neural-network-architectures-156e5bad51ba) para obtener un gran resumen de las diferentes arquitecturas ConvNet.

### Capas convolucionales

Una capa convolucional bidimensional asigna un *volumen* de entrada (es decir, un tensor de entrada tridimensional, por ejemplo, [ancho, altura, canales]) a un *volumen* de salida a través de un conjunto de filtros que se pueden aprender, que forman los parámetros de la capa. Cada filtro es pequeño espacialmente (a lo ancho y alto), pero se extiende a través de la profundidad total del volumen de entrada. (Por ejemplo: un filtro en la primera capa de un ConvNet puede tener un tamaño [5, 5, 3]). Durante el paso forward, hacemos convolucionar("deslizar") cada filtro a través del ancho y la altura del volumen de entrada y calculamos los productos de punto por elementos entre las entradas del filtro y la entrada en cualquier posición. A medida que deslizamos el filtro sobre el ancho y la altura del volumen de entrada, produciremos un mapa de activación bidimensional que proporciona las respuestas de ese filtro en cada posición espacial. Cada capa convolucional tendrá dicho conjunto de filtros, y cada uno de ellos producirá un mapa de activación bidimensional separado. Luego apilamos estos mapas de activación a lo largo de la dimensión de profundidad para producir el volumen de salida.

Mediante el uso de estos filtros que se asignan a un pequeño subvolumen de la entrada, podemos controlar en gran medida la explosión de parámetros que obtendríamos con una red feed-forward(totalmente conectada). Este **compartir de parámetros** en realidad también tiende a mejorar el rendimiento del modelo en entradas como imágenes naturales porque le proporciona al modelo cierta **invariancia de traducción** limitada. La invariancia de traducción significa que si la imagen (o una característica de la imagen) se traduce (mueve), el modelo no se verá afectado significativamente. ¡Piensa por qué es que ocurre esto!

La siguiente animación ilustra estas ideas, ¡asegúrese de comprenderlas!

![Animación de convolución](https://i.stack.imgur.com/FjvuN.gif)

Si el aspecto de compartir parámetros de las CNN todavía no está claro, considere el siguiente diagrama que compara una capa convolucional 1-D simplificada con una capa completamente conectada. El diagrama muestra cómo una entrada unidimensional $ \ mathbf {x} $ se asigna a una salida unidimensional $ \ mathbf {y} $ usando una capa completamente conectada y una capa de convolución, ambas sin parámetros de bias. Los colores de los bordes representan el valor de los parámetros de peso en las capas. Para la capa completamente conectada, el número de pesos es el producto de los tamaños de entrada y salida, en este caso, $ 6 \times 4 = 24 $. Por otro lado, el número de pesos en la capa convolucional depende solo del tamaño del filtro de la convolución, en este caso, $ 3 $, y es independiente de los tamaños de entrada y salida.

![Compartir de Pesos](https://i.imgur.com/gcmmZz4.png)


Los hiperparámetros de una capa convolucional son los siguientes:
* **Filtros** define el número de filtros en la capa
* **Tamaño del Kernel** define el ancho y la altura de los filtros (también llamados "kernels") en la capa. Tenga en cuenta que los kernels siempre tienen la misma profundidad que las entradas a la capa.
* **Stride** define el número de píxeles por los cuales movemos el filtro cuando lo "deslizamos" a lo largo del volumen de entrada. Por lo general, este valor sería 1, pero a veces también se usan valores de 2 y 3.
* **Padding** se refiere a la adición de píxeles de valor 0 a los bordes del volumen de entrada a lo largo de las dimensiones de ancho y alto. En Tensorflow puede establecer esto en "VALID", que esencialmente no añade padding o "SAME" que rellena la entrada de modo que el ancho y la altura de salida sean los mismos que la entrada (zero-padding).

Veamos un ejemplo ficticio muy simple para ver cómo los valores de los hiperparámetros afectan el tamaño de salida de una capa convolucional.

In [0]:
# Cree una "imagen" de color aleatorio de forma 10x10 con una profundidad de 3 (para rojo, verde y azul)
dummy_input = np.random.uniform(size=[10, 10, 3])
fig, ax = plt.subplots(1, 1)
plt.imshow(dummy_input)
ax.grid(False)
print('Input shape: {}'.format(dummy_input.shape))

Ahora ajuste los hiperparámetros con los controles deslizantes de la derecha y vea cómo cambia la forma de salida para una entrada [10, 10, 3].

In [0]:
#@title Parametros de una capa convolucional {run: "auto"}
filters = 2  #@param { type: "slider", min:0, max: 10, step: 1 }
kernel_size = 3 #@param { type: "slider", min:1, max: 10, step: 1 }
stride = 1 #@param { type: "slider", min:1, max: 3, step: 1 }

conv_layer = tf.keras.layers.Conv2D(
    filters=filters,
    kernel_size=kernel_size,
    strides=stride,
    padding="valid",
    input_shape=[10, 10, 3])

# Convierta la imagen en un tensor y agregue una dimensión de batch adicional 
# que la capa convolucional espera.
input_tensor = tf.convert_to_tensor(dummy_input[None, :, :, :])
convoluted = conv_layer(input_tensor)

print('Las dimensiones de salida son: {}'.format(list(convoluted.shape)[1:]))
print('El número de parametros es: {}'.format(conv_layer.count_params()))

Observe especialmente cómo el ancho y la altura de salida están relacionados con ```kernel_size``` y ```stride```, y cómo la profundidad de salida está relacionada con los ```filtros```.

**Pregunta:** ¿Puedes encontrar una fórmula para la forma de salida dada la forma de entrada, los hiperparámetros de la capa y suponiendo que no haya relleno?

### Lectura adicional opcional: creación de filtros complejos

Una de las razones por las que las CNN han tenido tanto éxito es su capacidad para construir filtros complejos componiendo filtros más simples. Por ejemplo, imagine un CNN de 5 capas que ha sido entrenado para detectar caras. Las primeras 4 capas son convolucionales y la última capa está completamente conectada y genera la predicción final (hay una cara o no). Podríamos encontrar que los filtros en cada capa de convolución distinguen las siguientes características:

1. líneas (horizontal, vertical, diagonal) y gradientes de color,
2. esquinas, círculos y otras formas simples, y texturas simples,
3. narices, bocas y ojos,
4. caras enteras.

¡La red neuronal ha aprendido a discenrnir objetos complejos como rasgos faciales e incluso caras enteras! La razón de esto es que cada capa sucesiva puede combinar los filtros de la capa anterior para detectar características cada vez más sofisticadas. El siguiente diagrama (adaptado de [este documento] (http://web.eecs.umich.edu/~honglak/icml09-ConvolutionalDeepBeliefNetworks.pdf)) muestra algunos ejemplos realmente geniales de este tipo de comportamiento. Las características de nivel inferior (mostradas arriba) detectan narices, ojos y bocas, en el caso de caras y ruedas, puertas y ventanas, para automóviles. Las características de nivel superior pueden detectar caras completas y automóviles.

![Imgur](https://i.imgur.com/653uIty.jpg)

Los diagramas en la página 7 de [este artículo clásico](https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf) muestran más ejemplos de este fenómeno y definitivamente vale la pena echarle un vistazo.

### (Max) Pooling
Una capa de pooling(agrupación) reduce el tamaño espacial de la representación. Hay diferentes razones por las cuales podemos querer hacer esto. Una es reducir el número de parámetros en la red. Imagine un convnet para el conjunto de datos MNIST. Si el tensor de características producido por la capa final conv/pool/relu fuera, por ejemplo, de tamaño 20x20 y tuviera 100 canales de características, la capa densa final tendría 20x20x100x10 = 400k parámetros. Sin embargo, si redujimos la muestra de esa capa a un tamaño espacial de 4x4, tendríamos solo 20k parámetros. ¡Una gran diferencia!

Otra razón es que queremos que las características posteriores (más profundas en la red) tengan *campos receptivos* más grandes (regiones de entrada que miran), para representar objetos y partes de objetos más grandes, por ejemplo. En particular, el stride de agrupación proporciona a las características posteriores campos receptivos mucho más grandes para que puedan combinar eficazmente características más pequeñas.

Una capa de pooling no tiene parámetros entrenables. Aplica alguna operación de agregación 2-D (generalmente un max(), pero otros como el promedio() también se pueden usar) a las regiones del volumen de entrada. Esto se hace independientemente para cada dimensión de profundidad de la entrada. Por ejemplo, una operación de max pooling de 2x2 con un tamaño de paso igual a 2,
 reduce la muestra de cada corte de profundidad de la entrada en 2 a lo largo del ancho y la altura.

Los hiperparámetros de una capa de pooling son los siguientes:
* **Tamaño del kernel** define cuántos valores se agregan juntos.
* **Stride** define el número de píxeles por los cuales movemos el filtro de agrupación al deslizarlo a lo largo de la entrada. Normalmente, este valor sería igual al tamaño del grupo.
* **Relleno** se refiere a la adición de píxeles de valor 0 a los bordes del volumen de entrada a lo largo de las dimensiones de ancho y alto. En Tensorflow puede establecer esto en "VÁLID", que no añade padding o "SAME" que rellena la entrada de modo que el ancho y la altura de salida sean los mismos que la entrada.

#### Pregunta
Haga 2x2 max-pooling a mano, con un stride de 2, y relleno "VÁLIDO", en la siguiente entrada 2D. ¿Cuál es el tamaño de la salida?

\begin{bmatrix}
  9 & 5 & 4 & 5 & 6 & 4 \\
  6 & 6 & 3 & 5 & 8 & 2 \\
  4 & 6 & 9 & 1 & 3 & 6 \\
  9 & 7 & 1 & 5 & 8 & 1 \\
  4 & 9 & 9 & 5 & 7 & 3 \\
  7 & 3 & 6 & 4 & 9 & 1 
\end{bmatrix}


¡Revela la celda de abajo haciendo doble clic y ejecutándola, para verificar tu respuesta cuando hayas terminado!

In [0]:
#@title Respuesta { display-mode: "form" }
X = np.array([[9, 5, 4, 5, 6, 4],
              [6, 6, 3, 5, 8, 2],
              [4, 6, 9, 1, 3, 6],
              [9, 7, 1, 5, 8, 1],
              [4, 9, 9, 5, 7, 3],
              [7, 3, 6, 4, 9, 1]])

max_pool_layer = tf.keras.layers.MaxPooling2D((2, 2), strides=2)
max_pool_layer(tf.convert_to_tensor(X[None, :, :, None])).numpy().squeeze()













































































































































### Lectura adicional opcional: Campos receptivos

Anteriormente mencionamos que una razón para realizar pooling es aumentar el tamaño de los campos receptivos de nuestras características. Echemos un vistazo más de cerca a lo que queremos decir con esto.

El siguiente diagrama muestra el campo receptivo efectivo de una "neurona" de salida en cada capa de unas pocas redes simples. Lo que el diagrama nos dice es cuántos de los valores de entrada tienen un efecto en cada valor de salida.

Podemos ver que en los primeros dos ejemplos, con capas convolucionales individuales, el campo receptivo es simplemente igual al tamaño del kernel.

Sin embargo, los siguientes dos ejemplos son un poco más interesantes. Aquí hemos aumentado drásticamente el tamaño del campo receptivo, sin un gran aumento en el número de parámetros, al apilar capas de convolución y agrupación. Lo interesante aquí es que al usar una capa de agrupación aumentamos nuestro tamaño de campo receptivo en un costo mucho menor (en el número de parámetros) que si simplemente hubiéramos aumentado los tamaños de kernel de nuestras capas de convolución.

Puedes leer más sobre los campos receptivos [aquí](https://medium.com/mlreview/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807).


![Campos receptivos](https://i.imgur.com/TjxEsG4.png)


## El conjunto de datos CIFAR10
Ahora que entendemos las capas convolucionales, de max-pooling y feed-forward, podemos combinarlas como bloques de construcción para construir un clasificador ConvNet para imágenes. Para esta práctica, utilizaremos el conjunto de datos de imagen en color CIFAR10 que consta de 50,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Seleccionaremos 10,000 imágenes del conjunto de entrenamiento para formar un conjunto de validación y visualizar algunas imágenes de ejemplo.

In [0]:
cifar = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()
cifar_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
# Seleccionar las últimas 10000 imágenes del conjunto de entrenamiento para formar un conjunto de validación
train_labels = train_labels.squeeze()
validation_images = train_images[-10000:, :, :]
validation_labels = train_labels[-10000:]
train_images = train_images[:-10000, :, :]
train_labels = train_labels[:-10000]

¿Cuáles son las formas y los tipos de datos de train_images y train_labels?



In [0]:
print('train_images.shape = {}, data-type = {}'.format(train_images.shape, train_images.dtype))
print('train_labels.shape = {}, data-type = {}'.format(train_labels.shape, train_labels.dtype))

print('validation_images.shape = {}, data-type = {}'.format(validation_images.shape, validation_images.dtype))
print('validation_labels.shape = {}, data-type = {}'.format(validation_labels.shape, validation_labels.dtype))

### Visualizar ejemplos del conjunto de datos
Ejecute la celda a continuación varias veces para ver varias imágenes. (Pueden verse un poco borrosos porque hemos volado las imágenes pequeñas).

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid('off')

  img_index = np.random.randint(0, 40000)
  plt.imshow(train_images[img_index])
  plt.xlabel(cifar_labels[train_labels[img_index]])

Un clasificador ConvNet
Finalmente, construimos una arquitectura convolucional simple para clasificar las imágenes CIFAR. Construiremos una versión mini de la arquitectura AlexNet, que consta de 5 capas convolucionales con max-pooling, seguidas de 3 capas completamente conectadas al final. Para investigar el efecto que tiene cada una de estas dos capas en la cantidad de parámetros, construiremos el modelo en dos etapas.

Primero, las capas convolucionales + max-pooling:

In [0]:
# Define the convolutinal part of the model architecture using Keras Layers.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(32, 32, 3), padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation=tf.nn.relu, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=192, kernel_size=(3, 3), activation=tf.nn.relu, padding='same'),
    tf.keras.layers.Conv2D(filters=192, kernel_size=(3, 3), activation=tf.nn.relu, padding='same'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation=tf.nn.relu, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
])


¿Cuántos parámetros hay en la parte convolucional de la arquitectura? Podemos inspeccionar esto fácilmente usando la función de resumen del modelo en Keras:

Primero hacerlo manualmente antes de usar summary

In [0]:
model.summary()

Ahora agregamos una parte completamente conectada. Tenga en cuenta que también agregamos "Dropout" después de la primera capa completamente conectada. Dropout es una técnica de regularización que elimina de forma aleatoria las conexiones entre las neuronas, y fue una de las innovaciones clave del artículo de AlexNet en 2012.

In [0]:
model.add(tf.keras.layers.Flatten())  # Aplanar(Flatten) "apretar" un volumen tridimensional en un solo vector.
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [0]:
model.summary()

### Lectura adicional opcional: esquemas de inicialización aleatoria

Es posible que te hayas preguntado qué valores estamos usando para los valores iniciales de los pesos y bias en nuestro modelo. La respuesta corta es que generalmente usamos inicialización aleatoria. En este caso, acabamos de utilizar los inicializadores Keras predeterminados para cada capa, que generalmente son suficientes.

La respuesta más larga es que el uso de números completamente aleatorios no siempre funciona mejor en la práctica y que hay una serie de esquemas de inicialización comunes (que están disponibles en la mayoría de los marcos de aprendizaje profundo como TensorFlow y Keras).

Consideremos algunos ejemplos:

 * Cuando se utiliza la activación de ReLU, es común inicializar los bias con pequeños números positivos porque esto alienta a las activaciones de ReLU a comenzar en el estado _on_, lo que ayuda a contrarrestar el _dying ReLU problem_ (problema de ReLU moribundo).

 * Cuanto más profundas sean las redes neuronales, más probable es que los gradientes se reduzcan hasta el punto en que desaparezcan o crezcan hasta el punto en que se desborden (los problemas de gradientes que desaparecen y explotan -_vanishing_ and _exploding_ gradients-). Para ayudar a combatir esto, podemos inicializar nuestros pesos para tener una escala apropiada (específica del modelo). Un método para hacer esto se llama inicialización [Xavier o Glorot] (http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf).

 * El esquema de inicialización _Xavier_ fue diseñado con las activaciones tradicionales Sigmoid y TanH en mente y no funciona tan bien para las activaciones ReLU. Una alternativa es la inicialización [He o Kaiming] (https://arxiv.org/pdf/1502.01852.pdf), que es una modificación de la inicialización de Xavier para las activaciones de ReLU.

 [Este blog] (http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization) entra en más detalles sobre la inicialización _He_ y _Xavier_. [La documentación de Keras] (https://keras.io/initializers/) enumera una serie de esquemas comunes.

###Visualizando el modelo

Construyamos un diagrama de flujo del modelo que hemos construido para ver cómo fluye la información entre las diferentes capas.

In [0]:
tf.keras.utils.plot_model(model, to_file='small_lenet.png', show_shapes=True, show_layer_names=True)
display.display(display.Image('small_lenet.png'))


### Entrenamiento y validación del modelo.
Se puede escribir el pipeline del conjunto de datos, la función de pérdida y el ciclo de entrenamiento para darle una buena apreciación de cómo funciona. Ahora, usaremos el bucle de entrenamiento integrado en Keras. Para conjuntos de datos simples y estándar como CIFAR, hacerlo de esta manera funcionará bien, ¡pero es importante saber qué sucede debajo  porque es posible que se deba escribir algunos o todos los pasos manualmente cuando trabaje con conjuntos de datos más complejos!

In [0]:
batch_size = 128
num_epochs = 10  # El número de épocas (pases completos a través de los datos) para entrenar

# Compilar el modelo agrega una función de pérdida, optimizador y métricas para rastrear durante el entrenamiento
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# La función fit permite adaptar el modelo compilado a algunos datos de entrenamiento
model.fit(x=train_images,
          y=train_labels,
          batch_size=batch_size,
          epochs=num_epochs,
          validation_data=(validation_images, validation_labels.astype(np.float32)))

print('Entrenamiento completo')

### Test de  performance
Finalmente, evaluamos qué tan bien funciona el modelo en el conjunto de prueba extendido

In [0]:
metric_values = model.evaluate(x=test_images, y=test_labels)

print('Final TEST performance')
for metric_value, metric_name in zip(metric_values, model.metrics_names):
  print('{}: {}'.format(metric_name, metric_value))

Tenga en cuenta que logramos aproximadamente un 80% de precisión del conjunto de entrenamiento, pero nuestra precisión de prueba es solo de alrededor del 67%. ¿Cuál crees que puede ser la razón de esto?

### Clasificando ejemplos
Ahora usamos nuestro modelo entrenado para clasificar una muestra de 25 imágenes del conjunto de prueba. Pasamos estas 25 imágenes a la función ```model.predict```, que devuelve una matriz dimensional [25, 10]. La entrada en la posición $ (i, j) $ de esta matriz contiene la probabilidad de que la imagen $ i $ pertenezca a la clase $ j $. Obtenemos la predicción más probable utilizando la función ```np.argmax``` que devuelve el índice de la entrada máxima a lo largo de las columnas. Finalmente, graficamos el resultado con la predicción y la probabilidad de predicción etiquetadas debajo de la imagen y la etiqueta verdadera en el lateral.

In [0]:
img_indices = np.random.randint(0, len(test_images), size=[25])
sample_test_images = test_images[img_indices]
sample_test_labels = [cifar_labels[i] for i in test_labels[img_indices].squeeze()]

predictions = model.predict(sample_test_images)
max_prediction = np.argmax(predictions, axis=1)
prediction_probs = np.max(predictions, axis=1)

In [0]:
plt.figure(figsize=(10,10))
for i, (img, prediction, prob, true_label) in enumerate(
    zip(sample_test_images, max_prediction, prediction_probs, sample_test_labels)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid('off')

  plt.imshow(img)
  plt.xlabel('{} ({:0.3f})'.format(cifar_labels[prediction], prob))
  plt.ylabel('{}'.format(true_label))


### Pregunta
¿Qué opinas de las predicciones del modelo? Mirando la confianza del modelo (la probabilidad asignada a la clase predicha), busque ejemplos de los siguientes casos:
1. El modelo era correcto con gran confianza.
2. El modelo era correcto con poca confianza.
3. El modelo era incorrecto con gran confianza.
4. El modelo era incorrecto con poca confianza.

¿Cuáles crees que serían los valores de pérdida (relativos) en esos casos?

### Lectura adicional opcional: incertidumbre en el aprendizaje profundo

Las redes neuronales profundas no se consideran muy buenas para estimar la incertidumbre en sus predicciones. Sin embargo, conocer la incertidumbre de su modelo puede ser muy importante para muchas aplicaciones. Por ejemplo, considere una herramienta de aprendizaje profundo para diagnosticar enfermedades, en este caso, ¡un falso negativo podría tener un impacto masivo en la vida de una persona! Realmente nos gustaría saber qué tan seguro es nuestro modelo en su predicción. Este es un campo incipiente de investigación, por ejemplo, consulte [este blog](https://www.cs.ox.ac.uk/people/yarin.gal/website/blog_3d801aa532c1ce.html) para obtener una buena introducción.

### Lectura adicional opcional: arquitecturas CNN

Decidir la arquitectura para una CNN, es decir, la combinación de convolución, pooling, capas densas y otras, puede ser complicado y, a menudo, puede parecer arbitrario. Además de eso, uno también tiene que tomar decisiones como qué tipo de agrupación, qué funciones de activación y qué tamaño de convolución usar, entre otras cosas. Para los nuevos y viejos practicantes de aprendizaje profundo, estas opciones pueden ser abrumadoras.Sin embargo, al examinar las arquitecturas exitosas existentes de CNN podemos aprender mucho sobre lo que funciona y lo que no. (Incluso podemos aplicar estas arquitecturas existentes a nuestros problemas, ya que muchas bibliotecas de aprendizaje profundo, como TensorFlow y Keras, las tienen [integradas](https://keras.io/applications/#available-models) e incluso es posible para ajustar modelos pre-entrenados a nuestro problema específico usando [transferencia de aprendizaje](https://cs231n.github.io/transfer-learning/).) [Este artículo](https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5) describe muchas de las arquitecturas CNN más exitosas de los últimos años, incluyendo [ ResNet](https://arxiv.org/abs/1512.03385), [Inception](https://arxiv.org/pdf/1512.00567v3.pdf) y [VGG](https://arxiv.org/pdf/1409.1556.pdf). Para obtener una descripción más detallada y técnica de estos modelos y más, consulte [estas diapositivas](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture9.pdf). La lectura de estos recursos debería brindarle información sobre por qué estas arquitecturas son exitosas, así como las mejores prácticas y las tendencias actuales para las CNN que lo ayudarán a diseñar sus propias arquitecturas.Por ejemplo, una de las prácticas que puede aprender es el uso de convoluciones 3x3. Notará que arquitecturas más antiguas como [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) utilizaron una gama de convoluciones de 7x7 hasta 3x3. Sin embargo, las arquitecturas más nuevas, como VGG y ResNet, utilizan convoluciones 3x3 casi exclusivamente. En resumen, la razón es que apilar convoluciones 3x3 le da el mismo campo receptivo que una convolución más grande pero con más no linealidad.Aquí hay algunas otras preguntas en las que puede pensar mientras investiga estas arquitecturas:
* ¿Por qué las arquitecturas modernas usan menos max-pooling?
* ¿Qué hace una convolución 1x1?
* ¿Qué es una conexión residual?

## Tus tareas
1. [**TODO**] Experimente con la arquitectura de red, intente cambiar los números, tipos y tamaños de capas, los tamaños de los filtros, el uso de diferentes padding, etc. ¿Cómo afectan estas decisiones al rendimiento del modelo? En particular, intente construir una red *totalmente convolucional*, sin capas de pooling(max-).
2. [**TODO**] Agregue NORMALIZACIÓN DE LOTES [documentación de Tensorflow](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization) y [trabajo de investigación ](http://proceedings.mlr.press/v37/ioffe15.pdf)) para mejorar la generalización del modelo.
3. [**AVANZADO**] Lea sobre las redes residuales ([documento original](https://arxiv.org/pdf/1512.03385.pdf)) y agregue **conexiones de acceso directo** a la arquitectura del modelo. Intente construir un simple "bloque residual" reutilizable como un [Modelo de Keras](https://www.tensorflow.org/api_docs/python/tf/keras/Model).
4. [**OPCIONAL**]. Visualice los filtros de las capas convolucionales usando Matplotlib. **SUGERENCIA**: puede recuperar una referencia a una capa individual del modelo secuencial de Keras llamando a ```model.get_layer (name)```, reemplazando "name" con el nombre de la capa. 

##Recursos adicionales

Aquí hay más información sobre ConvNets:

* Publicación del blog de Chris Colah en [Comprender las circunvoluciones](https://colah.github.io/posts/2014-07-Understanding-Convolutions/)
* [¿Cómo funcionan las redes neuronales convolucionales?](http://brohrer.github.io/how_convolutional_neural_networks_work.html)
* El [curso CS231n](https://cs231n.github.io/) que es un gran recurso que cubre casi todos los conceptos básicos de CNN
* [Bloques de construcción de interpretabilidad](https://distill.pub/2018/building-blocks/) (algunas visualizaciones de características CNN realmente geniales)